In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Anxiety_disorder"
cohort = "GSE119995"

# Input paths
in_trait_dir = "../../input/GEO/Anxiety_disorder"
in_cohort_dir = "../../input/GEO/Anxiety_disorder/GSE119995"

# Output paths
out_data_file = "../../output/preprocess/Anxiety_disorder/GSE119995.csv"
out_gene_data_file = "../../output/preprocess/Anxiety_disorder/gene_data/GSE119995.csv"
out_clinical_data_file = "../../output/preprocess/Anxiety_disorder/clinical_data/GSE119995.csv"
json_path = "../../output/preprocess/Anxiety_disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Exposure-induced changes of plasma mRNA expression levels in patients with panic disorder"
!Series_summary	"Anxiety disorders including panic disorders with or without agoraphobia are the most prevalent mental disorders. Exposure is a core technique within the framework of cognitive behavioral therapy to treat phobia- and anxiety-related symptoms. The primary aim of this study was to trace specific anxiety-related plasma gene expression changes of subjects with PD at three time points in order to identify biomarkers for acute anxiety states. In this intervention, the patient is exposed to highly feared and mostly avoided situations."
!Series_overall_design	"Blood samples from individuals with panic disorder (n=24) were drawn at three time points during exposure: baseline, 1 hour post-exposure and 24 hours after exposure-onset."
Sample Characteristics Dictionary:
{0: ['disease: panic disorder'], 1: ['tissue: blood plasma'], 2: ['Sex: female', 'Sex:

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Determine gene expression data availability
# From the background information, this appears to be a study on plasma mRNA expression
# which means it contains gene expression data
is_gene_available = True

# 2.1 Data Availability

# Trait (Anxiety disorder)
# All subjects have panic disorder (row 0), but this is a constant feature
# The timepoint (row 4) provides information about anxiety levels during exposure
# which can be used as a proxy for anxiety severity
trait_row = 4

# Age 
# Age information is not available in the sample characteristics
age_row = None

# Gender
# Gender information is available in row 2 (Sex)
gender_row = 2

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert timepoint to a binary trait for anxiety severity"""
    if value is None:
        return None
    
    # Handle non-string values
    if not isinstance(value, str):
        return None
    
    # Extract the value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Baseline (b1) represents pre-exposure (0)
    # Other timepoints (p24_1, pe1) represent post-exposure (1)
    if value == "b1":
        return 0  # baseline/pre-exposure
    elif value in ["p24_1", "pe1"]:
        return 1  # post-exposure (higher anxiety)
    else:
        return None

def convert_age(value):
    """Convert age to continuous value"""
    # Age is not available
    return None

def convert_gender(value):
    """Convert gender to binary value"""
    if value is None:
        return None
    
    # Handle non-string values
    if not isinstance(value, str):
        return None
    
    # Extract the value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # female=0, male=1
    if value.lower() == "female":
        return 0
    elif value.lower() == "male":
        return 1
    else:
        return None

# 3. Save Metadata
# Check trait data availability
is_trait_available = trait_row is not None

# Initial filtering on usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # We need to create a DataFrame that mimics the structure expected by geo_select_clinical_features
    # The sample characteristics dictionary shows the rows with their values
    sample_char_dict = {
        0: ['disease: panic disorder'], 
        1: ['tissue: blood plasma'], 
        2: ['Sex: female', 'Sex: male', 'Sex: not determined'], 
        3: ['medication: 0', 'medication: 1'], 
        4: ['timepoint: b1', 'timepoint: p24_1', 'timepoint: pe1'], 
        5: ['individual: 2', 'individual: 9', 'individual: 7', 'individual: 22', 'individual: 6', 
            'individual: 10', 'individual: 15', 'individual: 12', 'individual: 18', 'individual: 13', 
            'individual: 26', 'individual: 19', 'individual: 20', 'individual: 24', 'individual: 14', 
            'individual: 27', 'individual: 29', 'individual: 33', 'individual: 34', 'individual: 31', 
            'individual: 38', 'individual: 21', 'individual: 39', 'individual: 41']
    }
    
    # Convert to DataFrame with strings, not integers
    clinical_data = pd.DataFrame.from_dict(sample_char_dict, orient='index')
    
    # Extract clinical features
    clinical_data_selected = geo_select_clinical_features(
        clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview_result = preview_df(clinical_data_selected)
    print("Preview of selected clinical features:")
    print(preview_result)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_data_selected.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{0: [0.0, 0.0], 1: [1.0, 1.0], 2: [1.0, nan], 3: [nan, nan], 4: [nan, nan], 5: [nan, nan], 6: [nan, nan], 7: [nan, nan], 8: [nan, nan], 9: [nan, nan], 10: [nan, nan], 11: [nan, nan], 12: [nan, nan], 13: [nan, nan], 14: [nan, nan], 15: [nan, nan], 16: [nan, nan], 17: [nan, nan], 18: [nan, nan], 19: [nan, nan], 20: [nan, nan], 21: [nan, nan], 22: [nan, nan], 23: [nan, nan]}
Clinical data saved to ../../output/preprocess/Anxiety_disorder/clinical_data/GSE119995.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')

Gene data dimensions: 47290 genes × 72 samples


### Step 4: Gene Identifier Review

In [5]:
# Analyzing the gene identifiers
# The gene identifiers start with "ILMN_" which indicates they are Illumina probe IDs
# These are not standard human gene symbols and need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. From the output, we can see that 'ID' column in gene annotation contains Illumina IDs (ILMN_*)
# which matches the gene expression data identifiers, and 'Symbol' column contains the gene symbols
prob_col = 'ID'
gene_col = 'Symbol'

# 2. Get gene mapping dataframe from the annotation dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply gene mapping to convert probe-level measurements to gene expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print information about the mapping and resulting gene data
print(f"Mapped {len(gene_mapping)} probes to gene symbols")
print(f"Resulted in {len(gene_data)} unique genes")
print("First 10 gene symbols:")
print(gene_data.index[:10])
print(f"Gene data dimensions after mapping: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")


Mapped 44837 probes to gene symbols
Resulted in 21458 unique genes
First 10 gene symbols:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2',
       'A4GALT', 'A4GNT'],
      dtype='object', name='Gene')
Gene data dimensions after mapping: 21458 genes × 72 samples


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 normalized gene symbols: {normalized_gene_data.index[:5].tolist()}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Re-extract clinical data using the correct row indices and conversion functions from step 2
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Get background information and clinical data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Use the correct values and functions identified in step 2
trait_row = 4    # timepoint (b1=baseline vs others=post-exposure)
age_row = None   # No age data available 
gender_row = 2   # Sex information

def convert_trait(value):
    """Convert timepoint to a binary trait for anxiety severity"""
    if value is None:
        return None
    
    # Handle non-string values
    if not isinstance(value, str):
        return None
    
    # Extract the value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Baseline (b1) represents pre-exposure (0)
    # Other timepoints (p24_1, pe1) represent post-exposure (1)
    if value == "b1":
        return 0  # baseline/pre-exposure
    elif value in ["p24_1", "pe1"]:
        return 1  # post-exposure (higher anxiety)
    else:
        return None

def convert_gender(value):
    """Convert gender to binary value"""
    if value is None:
        return None
    
    # Handle non-string values
    if not isinstance(value, str):
        return None
    
    # Extract the value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # female=0, male=1
    if value.lower() == "female":
        return 0
    elif value.lower() == "male":
        return 1
    else:
        return None

# Extract clinical features with correct row indices and conversion functions
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 5. Determine if trait and demographic features are biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct final quality validation
is_trait_available = True  # We confirmed trait data is available in step 2
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains anxiety measurements (pre/post exposure) for panic disorder patients. Timepoints used as proxy for anxiety levels."
)

# 7. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for trait association studies, linked data not saved.")

Normalizing gene symbols...
Gene data shape after normalization: (20253, 72)
First 5 normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1']


Normalized gene data saved to ../../output/preprocess/Anxiety_disorder/gene_data/GSE119995.csv


Clinical data saved to ../../output/preprocess/Anxiety_disorder/clinical_data/GSE119995.csv
Linked data shape: (72, 20255)


Data shape after handling missing values: (72, 20255)
For the feature 'Anxiety_disorder', the least common label is '0.0' with 24 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Anxiety_disorder' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 18 occurrences. This represents 25.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Anxiety_disorder/cohort_info.json


Linked data saved to ../../output/preprocess/Anxiety_disorder/GSE119995.csv
